In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
path='https://raw.githubusercontent.com/elvajinj/Coursera_Capstone/master/uszips.csv'
df=pd.read_csv(path)
df.head()

zip      lat      lng       city state_id   state_name  zcta  parent_zcta  \
0  601  18.1800 -66.7522   Adjuntas       PR  Puerto Rico  True          NaN   
1  602  18.3607 -67.1752     Aguada       PR  Puerto Rico  True          NaN   
2  603  18.4544 -67.1220  Aguadilla       PR  Puerto Rico  True          NaN   
3  606  18.1672 -66.9383    Maricao       PR  Puerto Rico  True          NaN   
4  610  18.2903 -67.1224     Anasco       PR  Puerto Rico  True          NaN   

   population  density  county_fips county_name  \
0       18570    111.4        72001    Adjuntas   
1       41520    523.7        72003      Aguada   
2       54689    667.9        72005   Aguadilla   
3        6615     60.4        72093     Maricao   
4       29016    311.9        72011      Añasco   

                          all_county_weights  imprecise  military  \
0               {'72001':99.43,'72141':0.57}      False     False   
1                              {'72003':100}      False     False   
2                              {'72005':100}      False     False   
3  {'72093':94.88,'72121':1.35,'72153':3.78}      False     False   
4               {'72003':0.55,'72011':99.45}      False     False   

              timezone  
0  America/Puerto_Rico  
1  America/Puerto_Rico  
2  America/Puerto_Rico  
3  America/Puerto_Rico  
4  America/Puerto_Rico

In [4]:
df1=df.drop(['zcta','parent_zcta','population','density','county_fips','all_county_weights','imprecise','military','timezone'],axis=1)
df1.head()

zip      lat      lng       city state_id   state_name county_name
0  601  18.1800 -66.7522   Adjuntas       PR  Puerto Rico    Adjuntas
1  602  18.3607 -67.1752     Aguada       PR  Puerto Rico      Aguada
2  603  18.4544 -67.1220  Aguadilla       PR  Puerto Rico   Aguadilla
3  606  18.1672 -66.9383    Maricao       PR  Puerto Rico     Maricao
4  610  18.2903 -67.1224     Anasco       PR  Puerto Rico      Añasco

In [5]:
dff= df1[df.state_name == 'Texas']
dff.head()

zip      lat      lng        city state_id state_name county_name
25843  75001  32.9600 -96.8385     Addison       TX      Texas      Dallas
25844  75002  33.0897 -96.6075       Allen       TX      Texas      Collin
25845  75006  32.9619 -96.8970  Carrollton       TX      Texas      Dallas
25846  75007  33.0046 -96.8971  Carrollton       TX      Texas      Denton
25847  75009  33.3403 -96.7503      Celina       TX      Texas      Collin

In [6]:
address = 'Texas,US'

geolocator = Nominatim(user_agent="tx_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Texas are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Texas are 31.8160381, -99.5120986.


In [7]:
#only show Dallas
t_data = dff[dff['county_name'] == 'Dallas'].reset_index(drop=True)
t_data.head()

zip      lat      lng        city state_id state_name county_name
0  75001  32.9600 -96.8385     Addison       TX      Texas      Dallas
1  75006  32.9619 -96.8970  Carrollton       TX      Texas      Dallas
2  75019  32.9633 -96.9855     Coppell       TX      Texas      Dallas
3  75038  32.8746 -96.9976      Irving       TX      Texas      Dallas
4  75039  32.8875 -96.9422      Irving       TX      Texas      Dallas

In [8]:
address = 'Dallas, Texas'

geolocator = Nominatim(user_agent="tx_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dallas are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dallas are 32.7762719, -96.7968559.


In [9]:
# create map of Dallas using latitude and longitude values
map_dallas = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(t_data['lat'], t_data['lng'], t_data['city']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dallas)  
    
map_dallas

In [10]:
#utilizing the Foursquare API 
CLIENT_ID = 'SEV5BXUC4TOUUCBSB1PQ4D01ST3ZUF0HAWUKVHYOEPHNPO2R' # your Foursquare ID
CLIENT_SECRET = 'IQSUIELFYKX4MPKFPPOH2SHRUTC4I2QFYCG1XWRQWK1OFGI4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SEV5BXUC4TOUUCBSB1PQ4D01ST3ZUF0HAWUKVHYOEPHNPO2R
CLIENT_SECRET:IQSUIELFYKX4MPKFPPOH2SHRUTC4I2QFYCG1XWRQWK1OFGI4


In [11]:
#Let's explore the first city in our dataframe.
t_data.loc[0, 'city']

'Addison'

In [12]:
city_latitude = t_data.loc[0, 'lat'] # city latitude value
city_longitude = t_data.loc[0, 'lng'] # city longitude value

city_name = t_data.loc[0,'city'] # city name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))


Latitude and longitude values of Addison are 32.96, -96.8385.


In [13]:
#Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=SEV5BXUC4TOUUCBSB1PQ4D01ST3ZUF0HAWUKVHYOEPHNPO2R&client_secret=IQSUIELFYKX4MPKFPPOH2SHRUTC4I2QFYCG1XWRQWK1OFGI4&v=20180605&ll=32.96,-96.8385&radius=500&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cb3fa854c1f6726f1649246'},
 'response': {'headerLocation': 'Dallas',
  'headerFullLocation': 'Dallas',
  'headerLocationGranularity': 'city',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 32.9645000045, 'lng': -96.83314680376547},
   'sw': {'lat': 32.9554999955, 'lng': -96.84385319623452}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bed6223bac3c9b6a12cfee9',
       'name': 'La Spiga',
       'location': {'address': '4203 Linbdergh Dr',
        'lat': 32.958277970448925,
        'lng': -96.83741720119946,
        'labeledLatLngs': [{'label': 'display',
          'lat': 32.958277970448925,
          'lng': -96.83741720119946}],
        'distance': 216,
        'postalCode': '75001',
        'cc': 'US',
        'city': 

In [15]:
# function that extracts the category of the venue from all the citys in Dallas
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['city', 
                  'city latitude', 
                  'city longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
t_venues = getNearbyVenues(names=t_data['city'],
                                latitudes=t_data['lat'],
                                longitudes=t_data['lng']
                                  )

Addison
Carrollton
Coppell
Irving
Irving
Garland
Garland
Garland
Garland
Garland
Sachse
Grand Prairie
Grand Prairie
Grand Prairie
Irving
Irving
Irving
Irving
Richardson
Richardson
Rowlett
Rowlett
Cedar Hill
Desoto
Duncanville
Lancaster
Duncanville
Hutchins
Lancaster
Mesquite
Mesquite
Seagoville
Wilmer
Balch Springs
Mesquite
Sunnyvale
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas


In [17]:
print(t_venues.shape)
t_venues.head()

(813, 7)


city  city latitude  city longitude                      Venue  \
0  Addison          32.96        -96.8385                   La Spiga   
1  Addison          32.96        -96.8385  Cindi's N.Y. Delicatessen   
2  Addison          32.96        -96.8385      Enterprise Rent-A-Car   
3  Addison          32.96        -96.8385           Five Star Fabric   
4  Addison          32.96        -96.8385          Tone Shop Guitars   

   Venue Latitude  Venue Longitude       Venue Category  
0       32.958278       -96.837417               Bakery  
1       32.961864       -96.838873                Diner  
2       32.960673       -96.838102  Rental Car Location  
3       32.959985       -96.838228  Arts & Crafts Store  
4       32.959020       -96.839050          Music Store

In [18]:
t_venues.groupby('city').count()

city latitude  city longitude  Venue  Venue Latitude  \
city                                                                  
Addison                    9               9      9               9   
Balch Springs             17              17     17              17   
Carrollton                 3               3      3               3   
Coppell                    2               2      2               2   
Dallas                   627             627    627             627   
Desoto                     3               3      3               3   
Duncanville               10              10     10              10   
Garland                   18              18     18              18   
Grand Prairie              1               1      1               1   
Irving                    45              45     45              45   
Mesquite                  43              43     43              43   
Richardson                26              26     26              26   
Rowlett                    3               3      3               3   
Sachse                     3               3      3               3   
Sunnyvale                  3               3      3               3   

               Venue Longitude  Venue Category  
city                                            
Addison                      9               9  
Balch Springs               17              17  
Carrollton                   3               3  
Coppell                      2               2  
Dallas                     627             627  
Desoto                       3               3  
Duncanville                 10              10  
Garland                     18              18  
Grand Prairie                1               1  
Irving                      45              45  
Mesquite                    43              43  
Richardson                  26              26  
Rowlett                      3               3  
Sachse                       3               3  
Sunnyvale                    3               3

In [19]:
print('There are {} uniques categories.'.format(len(t_venues['Venue Category'].unique())))

There are 182 uniques categories.


In [20]:
t_venues=t_venues[t_venues['Venue Category'].str.contains('Restaurant')]
t_venues

city  city latitude  city longitude  \
18         Garland        32.8809        -96.6515   
21         Garland        32.8809        -96.6515   
36          Sachse        32.9720        -96.5808   
40          Irving        32.8260        -96.9655   
41          Irving        32.8260        -96.9655   
42          Irving        32.8260        -96.9655   
47          Irving        32.8470        -96.9569   
49          Irving        32.8470        -96.9569   
54          Irving        32.9205        -96.9859   
56          Irving        32.9205        -96.9859   
58          Irving        32.9205        -96.9859   
62          Irving        32.9205        -96.9859   
63          Irving        32.9205        -96.9859   
65          Irving        32.9205        -96.9859   
69          Irving        32.9205        -96.9859   
70          Irving        32.9205        -96.9859   
76          Irving        32.9205        -96.9859   
78          Irving        32.9205        -96.9859   
79          Irving        32.9205        -96.9859   
81      Richardson        32.9739        -96.7420   
82      Richardson        32.9739        -96.7420   
90      Richardson        32.9739        -96.7420   
95      Richardson        32.9739        -96.7420   
100     Richardson        32.9739        -96.7420   
102     Richardson        32.9488        -96.7102   
105     Richardson        32.9488        -96.7102   
113    Duncanville        32.6602        -96.9139   
129       Mesquite        32.8158        -96.6304   
130       Mesquite        32.8158        -96.6304   
131       Mesquite        32.8158        -96.6304   
133       Mesquite        32.8158        -96.6304   
134       Mesquite        32.8158        -96.6304   
135       Mesquite        32.8158        -96.6304   
136       Mesquite        32.8158        -96.6304   
137       Mesquite        32.8158        -96.6304   
138       Mesquite        32.8158        -96.6304   
139       Mesquite        32.8158        -96.6304   
147       Mesquite        32.8158        -96.6304   
148       Mesquite        32.8158        -96.6304   
152       Mesquite        32.8158        -96.6304   
157       Mesquite        32.8158        -96.6304   
160       Mesquite        32.8158        -96.6304   
162       Mesquite        32.8158        -96.6304   
166  Balch Springs        32.7189        -96.6184   
172  Balch Springs        32.7189        -96.6184   
175  Balch Springs        32.7189        -96.6184   
177  Balch Springs        32.7189        -96.6184   
178  Balch Springs        32.7189        -96.6184   
180  Balch Springs        32.7189        -96.6184   
181  Balch Springs        32.7189        -96.6184   
193         Dallas        32.7878        -96.7995   
194         Dallas        32.7878        -96.7995   
199         Dallas        32.7878        -96.7995   
200         Dallas        32.7878        -96.7995   
209         Dallas        32.7878        -96.7995   
210         Dallas        32.7878        -96.7995   
212         Dallas        32.7878        -96.7995   
214         Dallas        32.7878        -96.7995   
217         Dallas        32.7878        -96.7995   
218         Dallas        32.7878        -96.7995   
228         Dallas        32.7792        -96.8047   
234         Dallas        32.7792        -96.8047   
239         Dallas        32.7792        -96.8047   
242         Dallas        32.7792        -96.8047   
244         Dallas        32.7792        -96.8047   
246         Dallas        32.7792        -96.8047   
251         Dallas        32.7792        -96.8047   
253         Dallas        32.7792        -96.8047   
254         Dallas        32.7792        -96.8047   
261         Dallas        32.7792        -96.8047   
268         Dallas        32.7792        -96.8047   
284         Dallas        32.7792        -96.8047   
286         Dallas        32.7792        -96.8047   
290         Dallas        32.7792        -96.8047   
291         Dallas        32.7792      

In [21]:
# one hot encoding
t_onehot = pd.get_dummies(t_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
t_onehot['city'] = t_venues['city'] 

# move neighborhood column to the first column
fixed_columns = [t_onehot.columns[-1]] + list(t_onehot.columns[:-1])
t_onehot=t_onehot[fixed_columns]

In [22]:
t_onehot.shape

(171, 27)

In [23]:
#let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
t_grouped = t_onehot.groupby('city').mean().reset_index()
t_grouped

city  American Restaurant  Asian Restaurant  \
0  Balch Springs             0.000000          0.142857   
1         Dallas             0.099174          0.024793   
2    Duncanville             0.000000          0.000000   
3        Garland             0.500000          0.000000   
4         Irving             0.125000          0.000000   
5       Mesquite             0.187500          0.000000   
6     Richardson             0.000000          0.142857   
7         Sachse             0.000000          0.000000   

   Cajun / Creole Restaurant  Chinese Restaurant  Comfort Food Restaurant  \
0                   0.000000            0.142857                   0.0000   
1                   0.016529            0.024793                   0.0000   
2                   0.000000            0.000000                   0.0000   
3                   0.000000            0.000000                   0.0000   
4                   0.000000            0.062500                   0.0000   
5                   0.062500            0.062500                   0.0625   
6                   0.000000            0.142857                   0.0000   
7                   0.000000            1.000000                   0.0000   

   Fast Food Restaurant  French Restaurant  Greek Restaurant  \
0              0.428571           0.000000          0.000000   
1              0.190083           0.033058          0.008264   
2              1.000000           0.000000          0.000000   
3              0.000000           0.000000          0.000000   
4              0.125000           0.000000          0.000000   
5              0.125000           0.000000          0.000000   
6              0.000000           0.000000          0.000000   
7              0.000000           0.000000          0.000000   

   Hawaiian Restaurant  Indian Restaurant  Italian Restaurant  \
0               0.0000           0.000000            0.000000   
1               0.0000           0.008264            0.066116   
2               0.0000           0.000000            0.000000   
3               0.0000           0.000000            0.000000   
4               0.0625           0.187500            0.000000   
5               0.0000           0.000000            0.125000   
6               0.0000           0.000000            0.000000   
7               0.0000           0.000000            0.000000   

   Japanese Restaurant  Latin American Restaurant  Mediterranean Restaurant  \
0             0.000000                   0.000000                  0.000000   
1             0.041322                   0.024793                  0.008264   
2             0.000000                   0.000000                  0.000000   
3             0.000000                   0.000000                  0.000000   
4             0.000000                   0.000000                  0.062500   
5             0.000000                   0.000000                  0.000000   
6             0.000000                   0.000000                  0.142857   
7             0.000000                   0.000000                  0.000000   

   Mexican Restaurant  Middle Eastern Restaurant  New American Restaurant  \
0            0.142857                   0.000000                 0.000000   
1            0.206612                   0.008264                 0.041322   
2            0.000000                   0.000000                 0.000000   
3            0.500000                   0.000000                 0.000000   
4            0.062500                   0.000000                 0.062500   
5            0.250000                   0.000000                 0.000000   
6            0.285714                   0.142857                 0.000000   
7            0.000000                   0.000000                 0.000000   

   Restaurant  Salvadoran Restaurant  Seafood Restaurant  \
0    0.000000                 0.0000            0.142857   
1    0.033058                 0.0000            0.066116   
2    0.000000                 0.0000 

In [24]:
t_grouped.shape

(8, 27)

In [25]:
## Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in t_grouped['city']:
    print("----"+hood+"----")
    temp = t_grouped[t_grouped['city'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Balch Springs----
                  venue  freq
0  Fast Food Restaurant  0.43
1    Chinese Restaurant  0.14
2    Seafood Restaurant  0.14
3      Asian Restaurant  0.14
4    Mexican Restaurant  0.14


----Dallas----
                  venue  freq
0    Mexican Restaurant  0.21
1  Fast Food Restaurant  0.19
2   American Restaurant  0.10
3    Seafood Restaurant  0.07
4    Italian Restaurant  0.07


----Duncanville----
                  venue  freq
0  Fast Food Restaurant   1.0
1   American Restaurant   0.0
2    Mexican Restaurant   0.0
3       Thai Restaurant   0.0
4    Tex-Mex Restaurant   0.0


----Garland----
                 venue  freq
0  American Restaurant   0.5
1   Mexican Restaurant   0.5
2     Asian Restaurant   0.0
3      Thai Restaurant   0.0
4   Tex-Mex Restaurant   0.0


----Irving----
                             venue  freq
0                Indian Restaurant  0.19
1              American Restaurant  0.12
2             Fast Food Restaurant  0.12
3  Southern / Soul Food Re

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
#get top 5 venues
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['city'] = t_grouped['city']

for ind in np.arange(t_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

city 1st Most Common Venue  2nd Most Common Venue  \
0  Balch Springs  Fast Food Restaurant       Asian Restaurant   
1         Dallas    Mexican Restaurant   Fast Food Restaurant   
2    Duncanville  Fast Food Restaurant  Vietnamese Restaurant   
3        Garland   American Restaurant     Mexican Restaurant   
4         Irving     Indian Restaurant   Fast Food Restaurant   

  3rd Most Common Venue            4th Most Common Venue  \
0    Chinese Restaurant               Seafood Restaurant   
1   American Restaurant               Seafood Restaurant   
2       Thai Restaurant                 Asian Restaurant   
3   Japanese Restaurant                 Asian Restaurant   
4   American Restaurant  Southern / Soul Food Restaurant   

       5th Most Common Venue  
0         Mexican Restaurant  
1         Italian Restaurant  
2  Cajun / Creole Restaurant  
3  Cajun / Creole Restaurant  
4    South Indian Restaurant

In [28]:
#Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

t_grouped_clustering = t_grouped.drop('city', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 3, 4, 2, 3, 3, 3, 1], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

t_merged = t_data

# merge dallas_grouped with dallas_data to add latitude/longitude for each neighborhood
t_merged = t_merged.join(neighborhoods_venues_sorted.set_index('city'), on='city')

t_merged
# check the last columns!

zip      lat      lng           city state_id state_name county_name  \
0   75001  32.9600 -96.8385        Addison       TX      Texas      Dallas   
1   75006  32.9619 -96.8970     Carrollton       TX      Texas      Dallas   
2   75019  32.9633 -96.9855        Coppell       TX      Texas      Dallas   
3   75038  32.8746 -96.9976         Irving       TX      Texas      Dallas   
4   75039  32.8875 -96.9422         Irving       TX      Texas      Dallas   
5   75040  32.9277 -96.6201        Garland       TX      Texas      Dallas   
6   75041  32.8809 -96.6515        Garland       TX      Texas      Dallas   
7   75042  32.9139 -96.6749        Garland       TX      Texas      Dallas   
8   75043  32.8571 -96.5794        Garland       TX      Texas      Dallas   
9   75044  32.9626 -96.6532        Garland       TX      Texas      Dallas   
10  75048  32.9720 -96.5808         Sachse       TX      Texas      Dallas   
11  75050  32.7741 -97.0053  Grand Prairie       TX      Texas      Dallas   
12  75051  32.7276 -96.9944  Grand Prairie       TX      Texas      Dallas   
13  75052  32.6659 -97.0265  Grand Prairie       TX      Texas      Dallas   
14  75060  32.7960 -96.9545         Irving       TX      Texas      Dallas   
15  75061  32.8260 -96.9655         Irving       TX      Texas      Dallas   
16  75062  32.8470 -96.9569         Irving       TX      Texas      Dallas   
17  75063  32.9205 -96.9859         Irving       TX      Texas      Dallas   
18  75080  32.9739 -96.7420     Richardson       TX      Texas      Dallas   
19  75081  32.9488 -96.7102     Richardson       TX      Texas      Dallas   
20  75088  32.8923 -96.5491        Rowlett       TX      Texas      Dallas   
21  75089  32.9298 -96.5498        Rowlett       TX      Texas      Dallas   
22  75104  32.5804 -96.9662     Cedar Hill       TX      Texas      Dallas   
23  75115  32.5989 -96.8637         Desoto       TX      Texas      Dallas   
24  75116  32.6602 -96.9139    Duncanville       TX      Texas      Dallas   
25  75134  32.6201 -96.7830      Lancaster       TX      Texas      Dallas   
26  75137  32.6340 -96.9118    Duncanville       TX      Texas      Dallas   
27  75141  32.6400 -96.6946       Hutchins       TX      Texas      Dallas   
28  75146  32.5746 -96.7507      Lancaster       TX      Texas      Dallas   
29  75149  32.7700 -96.6149       Mesquite       TX      Texas      Dallas   
30  75150  32.8158 -96.6304       Mesquite       TX      Texas      Dallas   
31  75159  32.6075 -96.5409     Seagoville       TX      Texas      Dallas   
32  75172  32.6048 -96.6752         Wilmer       TX      Texas      Dallas   
33  75180  32.7189 -96.6184  Balch Springs       TX      Texas      Dallas   
34  75181  32.7270 -96.5552       Mesquite       TX      Texas      Dallas   
35  75182  32.8001 -96.5541      Sunnyvale       TX      Texas      Dallas   
36  75201  32.7878 -96.7995         Dallas       TX      Texas      Dallas   
37  75202  32.7792 -96.8047         Dallas       TX      Texas      Dallas   
38  75203  32.7463 -96.8030         Dallas       TX      Texas      Dallas   
39  75204  32.8021 -96.7888         Dallas       TX      Texas      Dallas   
40  75205  32.8366 -96.7962         Dallas       TX      Texas      Dallas   
41  75206  32.8316 -96.7703         Dallas       TX      Texas      Dallas   
42  75207  32.7866 -96.8212         Dallas       TX      Texas      Dallas   
43  75208  32.7537 -96.8399         Dallas       TX      Texas      Dallas   
44  75209  32.8488 -96.8259         Dallas       TX      Texas      Dallas   
45  75210  32.7713 -96.7460         Dallas       TX      Texas      Dallas   
46  75211  32.7349 -96.9030         Dallas       TX      Texas      Dallas   
47  75212  32.7814 -96.8792         Dallas       TX      Texas      Dallas   
48  75214  32.8287 -96.7454         Dallas       TX      Texas      Dallas   
49  75215  32.7507 -96.7579         Dallas       TX      Texas      Dallas   
50  75216  32.7116 -96.7837 

In [30]:
t_merged = t_merged.dropna()
t_merged.head()

zip      lat      lng     city state_id state_name county_name  \
3  75038  32.8746 -96.9976   Irving       TX      Texas      Dallas   
4  75039  32.8875 -96.9422   Irving       TX      Texas      Dallas   
5  75040  32.9277 -96.6201  Garland       TX      Texas      Dallas   
6  75041  32.8809 -96.6515  Garland       TX      Texas      Dallas   
7  75042  32.9139 -96.6749  Garland       TX      Texas      Dallas   

   Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
3             3.0     Indian Restaurant  Fast Food Restaurant   
4             3.0     Indian Restaurant  Fast Food Restaurant   
5             2.0   American Restaurant    Mexican Restaurant   
6             2.0   American Restaurant    Mexican Restaurant   
7             2.0   American Restaurant    Mexican Restaurant   

  3rd Most Common Venue            4th Most Common Venue  \
3   American Restaurant  Southern / Soul Food Restaurant   
4   American Restaurant  Southern / Soul Food Restaurant   
5   Japanese Restaurant                 Asian Restaurant   
6   Japanese Restaurant                 Asian Restaurant   
7   Japanese Restaurant                 Asian Restaurant   

       5th Most Common Venue  
3    South Indian Restaurant  
4    South Indian Restaurant  
5  Cajun / Creole Restaurant  
6  Cajun / Creole Restaurant  
7  Cajun / Creole Restaurant

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)*2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(t_merged['lat'], t_merged['lng'], t_merged['city'], t_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
#examine each cluster and determine the discriminating venue categories that distinguish each cluster
#Cluster 1

In [33]:
t_merged.loc[t_merged['Cluster Labels'] == 0, t_merged.columns[[0,3] + list(range(6, t_merged.shape[1]))]]

zip           city county_name  Cluster Labels 1st Most Common Venue  \
33  75180  Balch Springs      Dallas             0.0  Fast Food Restaurant   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
33      Asian Restaurant    Chinese Restaurant    Seafood Restaurant   

   5th Most Common Venue  
33    Mexican Restaurant

In [34]:
# Cluster 2

In [35]:
t_merged.loc[t_merged['Cluster Labels'] == 1, t_merged.columns[[0,3] + list(range(6, t_merged.shape[1]))]]

zip    city county_name  Cluster Labels 1st Most Common Venue  \
10  75048  Sachse      Dallas             1.0    Chinese Restaurant   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
10  Vietnamese Restaurant       Thai Restaurant      Asian Restaurant   

        5th Most Common Venue  
10  Cajun / Creole Restaurant

In [36]:
# Cluster 3

In [37]:
t_merged.loc[t_merged['Cluster Labels'] == 2, t_merged.columns[[0,3] + list(range(6, t_merged.shape[1]))]]

zip     city county_name  Cluster Labels 1st Most Common Venue  \
5  75040  Garland      Dallas             2.0   American Restaurant   
6  75041  Garland      Dallas             2.0   American Restaurant   
7  75042  Garland      Dallas             2.0   American Restaurant   
8  75043  Garland      Dallas             2.0   American Restaurant   
9  75044  Garland      Dallas             2.0   American Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
5    Mexican Restaurant   Japanese Restaurant      Asian Restaurant   
6    Mexican Restaurant   Japanese Restaurant      Asian Restaurant   
7    Mexican Restaurant   Japanese Restaurant      Asian Restaurant   
8    Mexican Restaurant   Japanese Restaurant      Asian Restaurant   
9    Mexican Restaurant   Japanese Restaurant      Asian Restaurant   

       5th Most Common Venue  
5  Cajun / Creole Restaurant  
6  Cajun / Creole Restaurant  
7  Cajun / Creole Restaurant  
8  Cajun / Creole Restaurant  
9  Cajun / Creole Restaurant

In [38]:
# Cluster 4

In [39]:
t_most=t_merged.loc[t_merged['Cluster Labels'] == 3, t_merged.columns[[0,3] + list(range(6, t_merged.shape[1]))]]
t_most

zip        city county_name  Cluster Labels 1st Most Common Venue  \
3   75038      Irving      Dallas             3.0     Indian Restaurant   
4   75039      Irving      Dallas             3.0     Indian Restaurant   
14  75060      Irving      Dallas             3.0     Indian Restaurant   
15  75061      Irving      Dallas             3.0     Indian Restaurant   
16  75062      Irving      Dallas             3.0     Indian Restaurant   
17  75063      Irving      Dallas             3.0     Indian Restaurant   
18  75080  Richardson      Dallas             3.0    Mexican Restaurant   
19  75081  Richardson      Dallas             3.0    Mexican Restaurant   
29  75149    Mesquite      Dallas             3.0    Mexican Restaurant   
30  75150    Mesquite      Dallas             3.0    Mexican Restaurant   
34  75181    Mesquite      Dallas             3.0    Mexican Restaurant   
36  75201      Dallas      Dallas             3.0    Mexican Restaurant   
37  75202      Dallas      Dallas             3.0    Mexican Restaurant   
38  75203      Dallas      Dallas             3.0    Mexican Restaurant   
39  75204      Dallas      Dallas             3.0    Mexican Restaurant   
40  75205      Dallas      Dallas             3.0    Mexican Restaurant   
41  75206      Dallas      Dallas             3.0    Mexican Restaurant   
42  75207      Dallas      Dallas             3.0    Mexican Restaurant   
43  75208      Dallas      Dallas             3.0    Mexican Restaurant   
44  75209      Dallas      Dallas             3.0    Mexican Restaurant   
45  75210      Dallas      Dallas             3.0    Mexican Restaurant   
46  75211      Dallas      Dallas             3.0    Mexican Restaurant   
47  75212      Dallas      Dallas             3.0    Mexican Restaurant   
48  75214      Dallas      Dallas             3.0    Mexican Restaurant   
49  75215      Dallas      Dallas             3.0    Mexican Restaurant   
50  75216      Dallas      Dallas             3.0    Mexican Restaurant   
51  75217      Dallas      Dallas             3.0    Mexican Restaurant   
52  75218      Dallas      Dallas             3.0    Mexican Restaurant   
53  75219      Dallas      Dallas             3.0    Mexican Restaurant   
54  75220      Dallas      Dallas             3.0    Mexican Restaurant   
55  75223      Dallas      Dallas             3.0    Mexican Restaurant   
56  75224      Dallas      Dallas             3.0    Mexican Restaurant   
57  75225      Dallas      Dallas             3.0    Mexican Restaurant   
58  75226      Dallas      Dallas             3.0    Mexican Restaurant   
59  75227      Dallas      Dallas             3.0    Mexican Restaurant   
60  75228      Dallas      Dallas             3.0    Mexican Restaurant   
61  75229      Dallas      Dallas             3.0    Mexican Restaurant   
62  75230      Dallas      Dallas             3.0    Mexican Restaurant   
63  75231      Dallas      Dallas             3.0    Mexican Restaurant   
64  75232      Dallas      Dallas             3.0    Mexican Restaurant   
65  75233      Dallas      Dallas             3.0    Mexican Restaurant   
66  75234      Dallas      Dallas             3.0    Mexican Restaurant   
67  75235      Dallas      Dallas             3.0    Mexican Restaurant   
68  75236      Dallas      Dallas             3.0    Mexican Restaurant   
69  75237      Dallas      Dallas             3.0    Mexican Restaurant   
70  75238      Dallas      Dallas             3.0    Mexican Restaurant   
71  75240      Dallas      Dallas             3.0    Mexican Restaurant   
72  75241      Dallas      Dallas             3.0    Mexican Restaurant   
73  75243      Dallas      Dallas             3.0    Mexican Restaurant   
74  75244      Dallas      Dallas             3.0    Mexican Restaurant   
75  75246      Dallas      Dallas             3.0    Mexican Restaurant   
76  75247      Dallas      Dallas             3.0    Mexican Restaurant   
77  75248      Dallas      Dall

In [40]:
# Cluster 5

In [41]:
t_merged.loc[t_merged['Cluster Labels'] == 4, t_merged.columns[[0,3] + list(range(6, t_merged.shape[1]))]]

zip         city county_name  Cluster Labels 1st Most Common Venue  \
24  75116  Duncanville      Dallas             4.0  Fast Food Restaurant   
26  75137  Duncanville      Dallas             4.0  Fast Food Restaurant   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
24  Vietnamese Restaurant       Thai Restaurant      Asian Restaurant   
26  Vietnamese Restaurant       Thai Restaurant      Asian Restaurant   

        5th Most Common Venue  
24  Cajun / Creole Restaurant  
26  Cajun / Creole Restaurant